In [1]:
# Auteur Kees van den Tempel, AI-labs BV, Tricht (kees@ai-labs.nl)
# Versie 1.0
# 1-8-2023

# This script generates synthetic data by accessing the OpenAI GPT3.5-Turbo Api. 16000 records, text data (Summarization and Q&A) from 1620 products and service of several Dutch governments for training LLM

# 1. It uses the list of all the Dutch products and services (State Government, Provinces, Waterschappen, Municipalities) 
# 2. This list is first unpivoted so it becomes a list of a product for one specific target (company or citizen): 1620 records
# 3. The for each record a Prompt is made, which is send to the OpenAI API
# 4. The result is a JSON string which is added to a dataframe
# 5. The dataframe is then written to disk

# Step 1 to 5 is done two times:
# A. to make a file with a context and a summary (1620) records
# B. to make a file with the 10 most asked question and the corresponding answer (16200 records)

# Both files contain an identifier so they can be joined to make one big dataset.

# This script runs for appr 2 days. To test this script in a shorter time (5 minutes), I introduced
# a Limit variable and set it to 4: So, it calls the OpenAI Api only 4 times

# Change this to at least 1620 to iterate through all the 1620 Dutch products and services
Limit = 4
# Fill in your own OpenAI API key: https://platform.openai.com/account/api-keys
MyApiKey = "<<YOUR-API-KEY>>"

import numpy as np
import pandas as pd
import openai
import time
import json

# Read the file with the products and service of all Dutch governments
pend = pd.read_csv("data/UPL-actueel.csv", decimal=".", sep=",")
print(pend.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   UniformeProductnaam  1240 non-null   object
 1   URI                  1195 non-null   object
 2   Rijk                 699 non-null    object
 3   Provincie            158 non-null    object
 4   Waterschap           99 non-null     object
 5   Gemeente             506 non-null    object
 6   Burger               988 non-null    object
 7   Bedrijf              678 non-null    object
 8   Dienstenwet          240 non-null    object
 9   SDG                  487 non-null    object
 10  Autonomie            208 non-null    object
 11  Medebewind           421 non-null    object
 12  Aanvraag             616 non-null    object
 13  Subsidie             78 non-null     object
 14  Melding              55 non-null     object
 15  Verplichting         120 non-null    object
 16  DigiDM

In [2]:
# The file UPL_Actueel is a pivoted list of records, and needs to be unpivoted.
# Pandas.met() Unpivots a DataFrame from wide to long format

#### FIRST TIME
# `pend` is your DataFrame
selected_columns = ['Rijk', 'Provincie', 'Waterschap', 'Gemeente']
id_vars = [col for col in pend.columns if col not in selected_columns]

# reshape the DataFrame
pend_melt1 = pend.melt(id_vars=id_vars, value_vars=selected_columns, var_name='Overheid', value_name='OA')

# filter the rows where Applicability is 'X'
pend_melt1 = pend_melt1[pend_melt1['OA'] == 'X']

# reset the index
pend_melt1.reset_index(drop=True, inplace=True)


#### SECOND TIME
# `pend_melt1` is your DataFrame
selected_columns = ['Burger', 'Bedrijf']
id_vars = [col for col in pend_melt1.columns if col not in selected_columns]

# reshape the DataFrame
pend_melt2 = pend_melt1.melt(id_vars=id_vars, value_vars=selected_columns, var_name='Klant', value_name='KA')

# filter the rows where Applicability is 'X'
pend_melt2 = pend_melt2[pend_melt2['KA'] == 'X']

# reset the index
pend_melt2.reset_index(drop=True, inplace=True)


#### THIRD TIME
# `pend_melt2` is your DataFrame
selected_columns = ['Aanvraag', 'Subsidie', 'Melding', 'Verplichting']
id_vars = [col for col in pend_melt2.columns if col not in selected_columns]

# reshape the DataFrame
pend_melt3 = pend_melt2.melt(id_vars=id_vars, value_vars=selected_columns, var_name='SoortPD', value_name='PA')

# filter the rows where Applicability is 'X'
pend_melt3 = pend_melt3[pend_melt3['PA'] == 'X']

# reset the index
pend_melt3.reset_index(drop=True, inplace=True)
pend_melt3.drop(['Autonomie', 'Medebewind', 'DigiDMacht', 'URI', 'Dienstenwet', 'SDG', 'DigiDMacht', 'Grondslaglink;;', 'Grondslag'], axis=1, inplace=True)


In [3]:
pend_melt3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563 entries, 0 to 1562
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   UniformeProductnaam  1563 non-null   object
 1   Grondslaglabel       1520 non-null   object
 2   Overheid             1563 non-null   object
 3   OA                   1563 non-null   object
 4   Klant                1563 non-null   object
 5   KA                   1563 non-null   object
 6   SoortPD              1563 non-null   object
 7   PA                   1563 non-null   object
dtypes: object(8)
memory usage: 97.8+ KB


In [4]:
# Printing unique values of the 'Overheid' column in the 'pend_melt3' dataframe
print(pend_melt3['Overheid'].unique())

# Creating a dictionary 'replacement_dict' to replace certain values in the 'Overheid' column
replacement_dict = {'Rijk': 'de Nederlandse Rijksoverheid', 'Provincie': 'een Nederlandse provincie', 
                    'Gemeente': 'een Nederlandse gemeente', 'Waterschap': 'een Nederlands waterschap'}

# Replacing values in the 'Overheid' column using the 'replacement_dict' dictionary
# The inplace=True parameter ensures the changes are applied to the original dataframe
pend_melt3['Overheid'].replace(replacement_dict, inplace=True)

# Printing unique values of the 'SoortPD' column in the 'pend_melt3' dataframe
print(pend_melt3['SoortPD'].unique())

# Creating a dictionary 'replacement_dict' to replace certain values in the 'SoortPD' column
replacement_dict = {'Aanvraag': 'een aanvraag doen', 'Subsidie': 'een subsidie aanvragen', 
                    'Melding': 'een melding maken', 'Verplichting': 'zijn verplichting nakomen volgens'}

# Replacing values in the 'SoortPD' column using the 'replacement_dict' dictionary
# The inplace=True parameter ensures the changes are applied to the original dataframe
pend_melt3['SoortPD'].replace(replacement_dict, inplace=True)

# Printing unique values of the 'Klant' column in the 'pend_melt3' dataframe
print(pend_melt3['Klant'].unique())

# Creating a dictionary 'replacement_dict' to replace certain values in the 'Klant' column
replacement_dict = {'Burger': 'burger', 'Bedrijf': 'bedrijf'}

# Replacing values in the 'Klant' column using the 'replacement_dict' dictionary
# The inplace=True parameter ensures the changes are applied to the original dataframe
pend_melt3['Klant'].replace(replacement_dict, inplace=True)


['Rijk' 'Provincie' 'Waterschap' 'Gemeente']
['Aanvraag' 'Subsidie' 'Melding' 'Verplichting']
['Burger' 'Bedrijf']


In [5]:
# Add an Identifier column
pend_melt3['ID'] = range(1, len(pend_melt3) + 1)
pend_melt3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563 entries, 0 to 1562
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   UniformeProductnaam  1563 non-null   object
 1   Grondslaglabel       1520 non-null   object
 2   Overheid             1563 non-null   object
 3   OA                   1563 non-null   object
 4   Klant                1563 non-null   object
 5   KA                   1563 non-null   object
 6   SoortPD              1563 non-null   object
 7   PA                   1563 non-null   object
 8   ID                   1563 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 110.0+ KB


In [6]:
def get_completion(prompt, model="gpt-3.5-turbo"):

    #This function takes a prompt and uses the OpenAI API to generate a chat completion response based on the given prompt.

    #Parameters:
    #    prompt (str): The input text prompt provided by the user.
    #    model (str): The name or identifier of the language model to be used for chat completion.
    #                 Default value is "gpt-3.5-turbo", which is a variant of the GPT-3.5 model.

    #Returns:
    #    str or None: The chat completion response generated by the model, or None if the API call fails.

    #Note:
    #    Before using this function, you need to have a valid OpenAI API key to authorize the API calls.
    #    The API key should be set as `openai.api_key` before invoking this function.

   
    openai.api_key = MyApiKey
    messages = [{"role": "user", "content": prompt}]
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0,  # this is the degree of randomness of the model's output
        )
        return response.choices[0].message["content"]
    except openai.OpenAIError as e:
        print(f"OpenAI API call failed: {e}")
        return None


In [7]:
#This script generates a prompt, interacts with an AI model (probably for language generation using OpenAI's GPT), 
#and then stores the generated result as a JSON string.
#The JSON string is added to a panda dataframe, which is written to disk in excel and csv
                                                     
# Create an empty list to store the JSON-strings
master_df1 = pd.DataFrame()

i = 0
for index, row in pend_melt3.iterrows():
    i=i+1
    
    # Create a prompt, that outputs an explanation of a Dutch Government product and the summary
    Prompt = "Ik ben een helpdesk medewerker en wil een tekst schrijven voor een website van de overheid voor burgers en bedrijven. "
    Prompt = Prompt + "Geef een uitleg en een samenvatting als een " + row['Klant'] + " uit Nederland " + row['SoortPD'] + " van een " + row['UniformeProductnaam'] + " bij " + row['Overheid'] + ". "
    Prompt = Prompt + "Geef de antwoorden op taalniveau B1. De uitleg is minimaal 25 zinnen. De samenvatting is maximaal 2 zinnen. "
    Prompt = Prompt + "Geef de resultaten in JSON formaat met de volgende indeling: \n"
    Prompt = Prompt + "{\n"
    Prompt = Prompt + "  \"Nummer\", \"" + str(row['ID']) + "\"\n"
    Prompt = Prompt + "  \"Uitleg\", \"uitleg\"\n"
    Prompt = Prompt + "  \"Samenvatting\", \"samenvatting\"\n"
    Prompt = Prompt + "}\n"

    # print the name of the product to see the progress of the routine
    print(str(i) + ": " + str(row['UniformeProductnaam']))
    if i==1:
        print(Prompt)

    JSONString = get_completion(Prompt)
    if JSONString is None:
        print(f"Skipping row {i} due to API error")
        continue  # Skip this iteration
    else:    
        try:
            # Convert JSON string to Python dict
            data_dict = json.loads(JSONString)

            # Convert Python dict to DataFrame
            temp_df = pd.json_normalize(data_dict)

            # Append DataFrame to master DataFrame using concat
            master_df1 = pd.concat([master_df1, temp_df], ignore_index=True)
        except json.JSONDecodeError:
            print(f"Error occurred at iteration {i}, skipping this iteration. JSON string was: {JSONString}")
            continue  # Skip this iteration

    # CHANGE Limit TO 10000!!
    if i==Limit: 
        break

    # Pause for 10 seconds, it seems that GPT-Api is performing better
    time.sleep(2)


1: aanvullende beurs kwijtschelding
Ik ben een helpdesk medewerker en wil een tekst schrijven voor een website van de overheid voor burgers en bedrijven. Geef een uitleg en een samenvatting als een burger uit Nederland een aanvraag doen van een aanvullende beurs kwijtschelding bij de Nederlandse Rijksoverheid. Geef de antwoorden op taalniveau B1. De uitleg is minimaal 25 zinnen. De samenvatting is maximaal 2 zinnen. Geef de resultaten in JSON formaat met de volgende indeling: 
{
  "Nummer", "1"
  "Uitleg", "uitleg"
  "Samenvatting", "samenvatting"
}

2: aanvullende inkomensvoorziening ouderen
3: aardbevingsbestendige nieuwbouw groningen vergoeding
4: adoptie- of pleegzorguitkering


In [8]:
master_df1.to_csv('data/NL_ProdenDienstenSum.csv', index=False)  # 'index=False' ensures that indices are not saved in the file
master_df1.to_excel('data/NL_ProdenDienstenSum.xlsx', index=False) 

In [9]:
# Create an empty list to store the dictionaries
master_df2 = pd.DataFrame()

i = 0
for index, row in pend_melt3.iterrows():
    i=i+1

    # Create a prompt, that outputs the the most asked questions and corresponding answers (in Dutch)
    Prompt = "Een " + row['Klant'] + " uit Nederland wil " + row['SoortPD'] + " van een " + row['UniformeProductnaam'] + " bij " + row['Overheid'] + ". "
    Prompt = Prompt + "Ik ben een helpdesk medewerker en wil een Q&A maken voor een website van de overheid voor bedrijven en burgers. "
    Prompt = Prompt + "Geef de 10 meest gestelde vragen en het bijbehorende antwoord die door een " + row['Klant'] + " gesteld worden over " + str(row['UniformeProductnaam']) + ". "
    Prompt = Prompt + "Geef het antwoord op taalniveau B1. Geef geen inleidende tekst en alleen de resultaten in JSON formaat met de volgende indeling: \n"
    Prompt = Prompt + "\n"
    Prompt = Prompt + "{\n"
    Prompt = Prompt + "  \"Nummer\"\n"
    Prompt = Prompt + "  \"Instantie\"\n"
    Prompt = Prompt + "  \"Vragen\": [\n"
    Prompt = Prompt + "    {\n"
    Prompt = Prompt + "      \"Vraag\"\n"
    Prompt = Prompt + "      \"Antwoord\"\n"
    Prompt = Prompt + "    },\n"
    Prompt = Prompt + "  ]\n"
    Prompt = Prompt + "}\n"
    # Prompt = Prompt + "####\n"
    Prompt = Prompt + "\n"
    Prompt = Prompt + "Het label 'Nummer' heeft de waarde '" + str(row['ID']) + "'. "
    Prompt = Prompt + "Het label 'Instantie' geeft de verantwoordelijke organisatie in Nederland voor " + str(row['UniformeProductnaam']) + "."

    # print the name of the product to see the progress of the routine
    print(str(i) + ": " + str(row['UniformeProductnaam']))
    if i==1:
        print(Prompt)

    JSONString = get_completion(Prompt)
    if JSONString is None:
        print(f"Skipping row {i} due to API error")
        continue  # Skip this iteration
    else:    
        try:
            # convert JSON string to Python dict
            data_dict = json.loads(JSONString)

            # convert Python dict to DataFrame
            temp_df = pd.json_normalize(data_dict, record_path =['Vragen'], meta=['Nummer', 'Instantie'])

            # append DataFrame to master DataFrame using concat
            master_df2 = pd.concat([master_df2, temp_df], ignore_index=True)
        except json.JSONDecodeError:
            print(f"Error occurred at iteration {i}, skipping this iteration. JSON string was: {JSONString}")
            continue  # Skip this iteration

    # CHANGE Limit TO 10000!!
    if i==Limit: 
        break

    # Pause for 10 seconds, it seems that OpenAI GPT-Api is performing better
    time.sleep(2)

1: aanvullende beurs kwijtschelding
Een burger uit Nederland wil een aanvraag doen van een aanvullende beurs kwijtschelding bij de Nederlandse Rijksoverheid. Ik ben een helpdesk medewerker en wil een Q&A maken voor een website van de overheid voor bedrijven en burgers. Geef de 10 meest gestelde vragen en het bijbehorende antwoord die door een burger gesteld worden over aanvullende beurs kwijtschelding. Geef het antwoord op taalniveau B1. Geef geen inleidende tekst en alleen de resultaten in JSON formaat met de volgende indeling: 

{
  "Nummer"
  "Instantie"
  "Vragen": [
    {
      "Vraag"
      "Antwoord"
    },
  ]
}

Het label 'Nummer' heeft de waarde '1'. Het label 'Instantie' geeft de verantwoordelijke organisatie in Nederland voor aanvullende beurs kwijtschelding.
2: aanvullende inkomensvoorziening ouderen
3: aardbevingsbestendige nieuwbouw groningen vergoeding
4: adoptie- of pleegzorguitkering


In [10]:
master_df2.to_csv('data/NL_ProdenDienstenQA.csv', index=False)  # 'index=False' ensures that indices are not saved in the file
master_df2.to_excel('data/NL_ProdenDienstenQA.xlsx', index=False) 